In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import datetime
import time

# 加载数据集并进行预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
data_path = '../data-unversioned/p1ch7/'
trainset = datasets.CIFAR10(root=data_path, train=True,
                            download=True, transform=transform_train)
testset = datasets.CIFAR10(root=data_path, train=False,
                           download=True, transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

# 定义超参数
batch_size = 128
epochs = 20
lr = 0.1
momentum = 0.9
weight_decay = 1e-4

# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Cifar-10的标签
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 模型定义-ResNet
model = ResNet18()
# model = ResNet18().to(device)

# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

print(datetime.datetime.now(), '开始运算')
#训练模型
for epoch in range(epochs):
    print('Epoch: {}，时间:{}'.format(epoch+1, datetime.datetime.now()))
    train_loss = 0.0
    train_acc = 0.0
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        # inputs, labels = inputs.to(device), labels.to(device)
        inputs, labels = inputs, labels
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_acc += torch.sum(preds == labels.data)

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)
    print('Epoch: {}, Train Loss: {:.4f}, Train Acc: {:.4f}'.format(epoch+1, train_loss, train_acc))

print(datetime.datetime.now(), '结束运算')

2023-06-06 16:38:59.400708 开始运算
Epoch: 1，时间:2023-06-06 16:38:59.400911
Epoch: 1, Train Loss: 2.0091, Train Acc: 0.2916
Epoch: 2，时间:2023-06-06 16:59:04.333987
Epoch: 2, Train Loss: 1.5548, Train Acc: 0.4254
Epoch: 3，时间:2023-06-06 17:22:12.827977
Epoch: 3, Train Loss: 1.3040, Train Acc: 0.5273
Epoch: 4，时间:2023-06-06 17:45:38.085810
Epoch: 4, Train Loss: 1.0905, Train Acc: 0.6118
Epoch: 5，时间:2023-06-06 18:09:21.840667
Epoch: 5, Train Loss: 0.9304, Train Acc: 0.6705
Epoch: 6，时间:2023-06-06 18:33:01.954584
Epoch: 6, Train Loss: 0.8168, Train Acc: 0.7125
Epoch: 7，时间:2023-06-06 18:56:56.716948


In [ ]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>
        print("Accuracy {}: {:.2f}".format(name , correct / total))

print(datetime.datetime.now(), '开始运算')
validate(model, train_loader, val_loader)
print(datetime.datetime.now(), '结束运算')